In [14]:
from sympy.interactive import printing
printing.init_printing(use_latex = True)

import numpy as np
import sympy as sp

from scipy import integrate
from scipy.optimize import fsolve

import csv

T, gamma, epsilon, mu, E_g, K= sp.symbols('T, gamma, epsilon, mu, E_g, K')
#A, B = sp.symbols('A, B')

hk = sp.Symbol('h_k')
hk_value = 6.5821195E-16 # eV*s

kB = sp.Symbol('k_B')
kB_value = 8.6173333E-5 # eV/K

m_electron = 9.10938356E-31 #kg??

m_l = sp.Symbol('m_l')
m_l_value = 0.24

m_t = sp.Function('m_t')
m_t = (0.02459+8.659341E-5*T)

m_d0 = sp.Function('m_d0')
m_d0 = ((m_l * m_t**2)**(1/3))*m_electron

e = sp.Symbol('e')
e_value = ((1.4399764)**(1/2))*(10**(-7/2)) # in Lorenz-Heavyside units (eV*cm)**(1/2)

e_coulomb = sp.Symbol('e_coulomb')
e_coulomb_value = 1.602176634E-19 #Coulomb

C_l = sp.Symbol('C_l')
C_l_value = 0.71E11 #N/m2

E_ac = sp.Symbol('E_ac')
E_ac_value = 15 # 15 eV

a = sp.Symbol('a')
a_value = 6.461E-10 #m

rho = sp.Symbol('rho')
#rho_value = 8240 #kg/m3

hkomega0 = sp.Symbol('hkomega0')
hkomega0_value = 0.0136 #eV ???

E_oc = sp.Symbol('E_oc')
E_oc_value = E_ac_value # = 15 eV

epsilon_oo = sp.Symbol('epsilon_oo')
epsilon_oo_value = 32.6

epsilon_0 = sp.Symbol('epsilon_0')
epsilon_0_value = 400

#Carrier concentration functions
#m_t -> m_dprim -> n_multiplier -> n_integral

def n(mu_value, gamma_value, T_value, Eg_value):
    mu_value = mu_value
    gamma_value = gamma_value
    T_value = T_value
    Eg_value = Eg_value    

    n_multiplier = sp.Function('n_multiplier')
    n_multiplier = sp.sqrt(2)/(sp.pi**2 * hk**3) * gamma * m_d0**(3/2) 
    #display(n_multiplier)
    
    fermi = sp.Function('fermi')
    fermi = 1/ (1 + sp.exp((epsilon - mu)/(kB*T)))
    #display(fermi)

    n_integral = sp.Function('n_integral')
    n_integral = (sp.sqrt(epsilon * (1 + epsilon/E_g))) * (1 + 2*epsilon/E_g) * fermi * n_multiplier
    #display(n_integral)
    n_integral = n_integral.subs(hk, hk_value).subs(kB, kB_value).subs(m_l, m_l_value)

    n_integral = n_integral.subs(gamma, gamma_value).subs(T, T_value).subs(E_g, Eg_value).subs(mu, mu_value)
    #display(n_integral)
    n_integral = n_integral * (1.60217662E-19)**(-3/2) * (1.0E-6) # cm-3

    lambda_n_integral = sp.lambdify(epsilon, n_integral, 'numpy')
    result = integrate.quad(lambda_n_integral, 0, np.inf)
    return result[0]

def t_total(epsilon_value, T_value, Eg_value, mu_value, gamma_value, K_value):
    
    epsilon_value = epsilon_value
    T_value = T_value
    Eg_value = Eg_value
    mu_value = mu_value
    gamma_value = gamma_value
    K_value = K_value

    #test values---------------------
    #T_value = 600
    #Eg_value = 0.358
    #epsilon_value = 0.1
    #mu_value = 0.091
    #gamma_value = 4
    #N_v_value = # vacancy density

    #Relaxation times

    #----acoustic and optic framework---------------
    A = sp.Function('A')
    A = (epsilon/E_g) * (1 - K) / (1+2*epsilon/E_g)

    B = sp.Function('B')
    B = 8 * epsilon / E_g * (1 + epsilon/E_g) * K / (3*(1 + 2*epsilon/E_g)**2)

    t_ac_opt_frame = sp.Function('t_ac_opt_frame')
    t_ac_opt_frame = (epsilon + (epsilon**2)/E_g)**(-1/2) / ((1+2*epsilon/E_g)*(((1-A)**2)-B))

    #acoustic phonons--------------
    t_0a = sp.Function('t_0a')
    t_0a = (2 * sp.pi * hk**4 * C_l) / (E_ac**2 * kB * T * (2*m_d0)**(3/2))


    t_acoustic = sp.Function('t_acoustic')
    t_acoustic = t_0a*t_ac_opt_frame
    t_acoustic = t_acoustic.subs(kB, kB_value).subs(hk, hk_value).subs(C_l, C_l_value).subs(E_ac, E_ac_value).subs(m_l, m_l_value)
    t_acoustic = t_acoustic * (1.60217662E-19)**(1/2)

    t_acoustic = t_acoustic.subs(E_g, Eg_value).subs(T, T_value).subs(epsilon, epsilon_value).subs(K, K_value) 

    #optical phonons------------
    t_0o = sp.Function('t_0o')
    t_0o = (2 * hk**2 * a**2 * rho * (hkomega0)**2) / (sp.pi * E_oc**2 * kB * T * (2*m_d0)**(3/2))


    t_optic = sp.Function('t_optic')
    t_optic = t_0o*t_ac_opt_frame
    t_optic = t_optic.subs(kB, kB_value).subs(hk, hk_value).subs(hkomega0, hkomega0_value).subs(E_oc, E_oc_value).subs(m_l, m_l_value)
    t_optic = t_optic * (1.60217662E-19)**(1/2)

    t_optic = t_optic.subs(a, a_value).subs(rho, rho_value).subs(E_g, Eg_value).subs(T, T_value).subs(epsilon, epsilon_value).subs(K, K_value)
    
    #alloy scattering of elecrons-----------
    
    #Insert formula here
    
    
    #polar optical phonons----------------
    k_sq = sp.Function('k_sq')
    k_sq = 2*m_d0*(epsilon + (epsilon**2)/E_g) / (hk**2)

    fermi = sp.Function('fermi')
    fermi = 1/ (1 + sp.exp((epsilon - mu)/(kB*T)))

    diff_fermi = sp.Function('diff_fermi')
    diff_fermi = sp.diff(fermi, epsilon)

    fermi_int = sp.Function('fermi_int')
    fermi_int = diff_fermi * (epsilon*(1+epsilon/E_g))**(1/2) * (1+2*epsilon/E_g) *(-1)

    r_0 = sp.Function('r_0')
    r_0 = 2**(3/2) * e**2 * gamma * m_d0**(3/2) / (sp.pi * hk**3 * epsilon_oo) * fermi_int
    r_0 = r_0.subs(kB, kB_value).subs(hk, hk_value).subs(epsilon_oo, epsilon_oo_value).subs(e, e_value).subs(m_l, m_l_value)
    r_0 = r_0.subs(mu, mu_value).subs(E_g, Eg_value).subs(T, T_value).subs(gamma, gamma_value)

    lam_r_0 = sp.lambdify(epsilon, r_0, 'numpy')
    r_0_value = integrate.quad(lam_r_0, 0, 10)[0]

    delta = sp.Function('delta')
    delta = 2**(-2) * k_sq**(-1) 
    delta = delta * r_0_value * 0.01 * (1.60217662E-19)**(-1/2)
    F = sp.Function('F')
    F = 1 - delta * sp.ln(1 + 1/delta) - (1 - 2*delta + 2 * (delta**2) * sp.ln(1 + 1/delta)) * (2*epsilon/E_g * (1 + epsilon/E_g)) / ((1 + 2*epsilon/E_g)**2)

    t_polar_optic = sp.Function('t_polar_optic')
    t_polar_optic = (hk**2) * (1/F)*(epsilon + (epsilon**2) / E_g)**(1/2) / (e**2 * (2 * m_d0)**(1/2) * kB * T * ((1/epsilon_oo)-(1/epsilon_0)) * (1 + 2 * epsilon / E_g))
    t_polar_optic = t_polar_optic.subs(e, e_value).subs(kB, kB_value).subs(hk, hk_value).subs(epsilon_oo, epsilon_oo_value).subs(epsilon_0, epsilon_0_value).subs(m_l, m_l_value)
    t_polar_optic = t_polar_optic.subs(epsilon, epsilon_value).subs(E_g, Eg_value).subs(T, T_value)
    t_polar_optic = t_polar_optic * 100 / (6.24E18)**(1/2)

    #-------------------------------------

    t_total = ((t_acoustic.evalf()**(-1)) + (t_optic.evalf()**(-1)) + (t_polar_optic.evalf()**(-1)) + 0 + 0 )**(-1)

    #print(t_acoustic.evalf()**(-1)/1.0E11)
    #print(t_optic.evalf()**(-1)/1.0E11)
    #print(t_polar_optic.evalf()**(-1)/1.0E11)
    #print(t_total*1.0E13)
    
    return(t_total)

#t_total(0.1, 600, 0.358, 0.091, 4, 1)

def SIGMA(epsilon_value, T_value, Eg_value, mu_value, gamma_value, K_value):
    
    epsilon_value = epsilon_value
    T_value = T_value
    Eg_value = Eg_value
    mu_value = mu_value
    gamma_value = gamma_value
    K_value = K_value
    
    
    sigma_func = sp.Function('sigma_func')
    sigma_func = ((2/3) * gamma * (2*m_d0)**(1/2) / (sp.pi**2 * hk**3))
    ####_____CHANGED LINE_______
    sigma_func = sigma_func * (( epsilon * (1 + epsilon / E_g))**(3/2) * (1 + 2 * epsilon / E_g)**(-1))
    ####________________________
    sigma_func = sigma_func * t_total(epsilon_value, T_value, Eg_value, mu_value, gamma_value, K_value)
    sigma_func = sigma_func.subs(m_l, m_l_value).subs(hk, hk_value)
    sigma_func = sigma_func.subs(epsilon, epsilon_value).subs(E_g, Eg_value).subs(gamma, gamma_value).subs(T, T_value)
    
    return sigma_func.evalf()

def integral(T_value, Eg_value, mu_value, gamma_value, K_value, power_value):
    """
    Returns the value of integral in:
    1 / ( eV * s * cm ) for power == 0
    1 / ( s * cm ) for power == 1
    eV / ( s * cm) for power == 2
    """
    T_value = T_value
    Eg_value = Eg_value
    mu_value = mu_value
    gamma_value = gamma_value
    K_value = K_value
    power_value = power_value
    
    fermi = sp.Function('fermi')
    fermi = 1/ (1 + sp.exp((epsilon - mu)/(kB*T)))

    diff_fermi = sp.Function('diff_fermi')
    diff_fermi = sp.diff(fermi, epsilon)
    
    diff_fermi = diff_fermi.subs(kB, kB_value)
    diff_fermi = diff_fermi.subs(mu, mu_value).subs(T, T_value)
    
    lam_diff_fermi = sp.lambdify(epsilon, diff_fermi, 'numpy')
    lam_sigma = lambda epsilon: SIGMA(epsilon, T_value, Eg_value, mu_value, gamma_value, K_value)
    
    lam_final_integral = lambda epsilon: (-1) * lam_diff_fermi(epsilon) * lam_sigma(epsilon) * (epsilon - mu_value)**power_value
    result = integrate.quad(lam_final_integral, 0, 8)
    return result[0] * 0.01 * (6.24E18)**(1/2) 
    
def el_cond(T_value, Eg_value, mu_value, gamma_value, K_value):
    """
    Returns electrical conductivity in S/cm
    """
    T_value = T_value
    Eg_value = Eg_value
    mu_value = mu_value
    gamma_value = gamma_value
    K_value = K_value
    power_value = 0
    
    return integral(T_value, Eg_value, mu_value, gamma_value, K_value, power_value)/(6.24E18)


def Seeb(T_value, Eg_value, mu_value, gamma_value, K_value):
    """
    Returns Seebeck coefficint in (1E-6) V/K
    """
    T_value = T_value
    Eg_value = Eg_value
    mu_value = mu_value
    gamma_value = gamma_value
    K_value = K_value
    power_value_1 = 1
    power_value_0 = 0
    
    integrals = integral(T_value, Eg_value, mu_value, gamma_value, K_value, power_value_1) / integral(T_value, Eg_value, mu_value, gamma_value, K_value, power_value_0)
    return (1/T_value) * integrals * (6.24E18*1.602E-19*1.0E6)

def el_thermal_cond(T_value, Eg_value, mu_value, gamma_value, K_value):
    """
    Returns great electronic thermal conductivity in W/m/K
    """
    T_value = T_value
    Eg_value = Eg_value
    mu_value = mu_value
    gamma_value = gamma_value
    K_value = K_value
    power_value = 2
    
    return (1/T_value) * integral(T_value, Eg_value, mu_value, gamma_value, K_value, power_value) * (1.602E-19) *100

def Calculate_band_properties(Temp, bandgap, chem_pot, degeneracy, Kparam):
    Temp = Temp
    bandgap = bandgap #calculated for L band, 0.358eV for Sigma band
    chem_pot = chem_pot
    degeneracy = degeneracy #4 for L band, 12 for Sigma band
    Kparam = Kparam #1 for n-type, 1.5 for p-type
    
    sigma = el_cond(Temp, bandgap, chem_pot, degeneracy, Kparam)
    alfa = Seeb(Temp, bandgap, chem_pot, degeneracy, Kparam)
    kappa_e = el_thermal_cond(Temp, bandgap, chem_pot, degeneracy, Kparam) - Temp * sigma * (alfa**2) * 1.0E-10
    print(degeneracy)
    
    return (sigma, alfa, kappa_e)

#print(Calculate_band_properties(600, 0.358, 0.112, 4, 1.5))

# Spectrum calculations Ahmad style 3 bandmodel
hh_Eg = 0.41

def Bandgap_L(T_value, Sn_content):
    T_value = T_value #K
    Sn_content = Sn_content #[0...1]
    
    bandgap = 0.19-0.543*Sn_content + (3.2E-4) * T_value
    if bandgap < hh_Eg: return bandgap
    else: return hh_Eg

    
def update_rho(Sn_content):
    Sn_content = Sn_content

    global rho_value
    rho_value = (1-Sn_content)*8240 + Sn_content * 6440
    

def Calculate_mu(T_value, Sn_content, N_effective):
    T_value = T_value
    Sn_content = Sn_content
    N_effective = N_effective
    
    def function(mu_value, T_value, Sn_content, N_effective):
        """
        N_effective is the effective number of donors (+) and acceptors (-)
        (N_effective = N_donors - N_acceptors)
        """
        mu_value = mu_value
        T_value = T_value
        Sn_content = Sn_content
        N_effective = N_effective
        
        electrons = n(mu_value, 4, T_value, Bandgap_L(T_value, Sn_content))
        light_holes = n(-mu_value-Bandgap_L(T_value, Sn_content), 4, T_value, Bandgap_L(T_value, Sn_content))#-deltaE_L(T_value, Sn_content, hh_Eg)
        heavy_holes = n(-mu_value-hh_Eg, 12, T_value, hh_Eg)#-deltaE_sigma(T_value, Sn_content, hh_Eg)
        
        return  electrons - light_holes - heavy_holes - N_effective
    
    final_mu_value = fsolve(function, 0.001, args=(T_value, Sn_content, N_effective))
    
    electrons = n(final_mu_value, 4, T_value, Bandgap_L(T_value, Sn_content))
    light_holes = n(-final_mu_value-Bandgap_L(T_value, Sn_content), 4, T_value, Bandgap_L(T_value, Sn_content))#-deltaE_L(T_value, Sn_content, hh_Eg)
    heavy_holes = n(-final_mu_value-hh_Eg, 12, T_value, hh_Eg) #-deltaE_sigma(T_value, Sn_content, hh_Eg)
    
    return (final_mu_value[0], electrons, light_holes, heavy_holes)

    
#def Point_p_type(Temp, Sn_content, N_effective):
def Point_n_type(*data):
    """
    Evaluation according to Dymitrev
    result lines legend:
    res_line_0: Temp, Sn_content, N_effective
    res_line_1: chem_pot, light hole concentr, heavy hole concentr
    res_line_2: electrons: sigma, alfa, ke
    res_line_3: light holes: sigma, alfa, ke
    res_line_4: heavy holes: sigma, alfa, ke
    res_line_5: A, B, C, additional coefficients to ke
    res_line_6: sigma, alfa, ke, total values
    """
    
    #Temp = Temp
    #Sn_content = Sn_content
    #N_effective = N_effective
    
    Temp, Sn_content, N_effective = data
    
    update_rho(Sn_content)
    
    res_line_0 = (Temp, Sn_content, N_effective)
    print(res_line_0)
    
    res_line_1 = Calculate_mu(Temp, Sn_content, N_effective)
    print(res_line_1)
    chem_pot = res_line_1[0]
    
    #electrons
    res_line_2 = Calculate_band_properties(Temp, Bandgap_L(Temp, Sn_content), chem_pot, 4.0, 1.0)
    print(res_line_2)
    #light holes
    res_line_3 = Calculate_band_properties(Temp, Bandgap_L(Temp, Sn_content), -chem_pot-Bandgap_L(Temp, Sn_content), 4.0, 4.5)#deltaE_L(Temp, Sn_content, hh_Eg)-
    print(res_line_3)
    #heavy holes
    res_line_4 = Calculate_band_properties(Temp, hh_Eg, -chem_pot-hh_Eg, 12.0, 4.5)#-deltaE_sigma(Temp, Sn_content, hh_Eg)
    print(res_line_4)
    
    Tot_el_cond = res_line_2[0] + res_line_3[0] + res_line_4[0]
    
    Tot_Seeb = (-res_line_2[1]*res_line_2[0] + res_line_3[1]*res_line_3[0] + res_line_4[1]*res_line_4[0])/Tot_el_cond
    
    def ke_sub(T, el1, el2, S1, S2):
        T=T
        el1 = el1
        el2 = el2
        S1 = S1
        S2 = S2
        return T*el1*el2 / (el1+el2) * ((S1+S2)**2) * (1.0E-10) #W/m/K
    
    #electron+light_holes
    A = ke_sub(Temp, res_line_2[0], res_line_3[0], -res_line_2[1], res_line_3[1])
    #electron+heavy_holes
    B = ke_sub(Temp, res_line_2[0], res_line_4[0], -res_line_2[1], res_line_4[1])
    #light_holes-heavy_holes
    C = ke_sub(Temp, res_line_3[0], res_line_4[0], res_line_3[1], -res_line_4[1])
    
    Tot_ke = res_line_2[2] + res_line_3[2] + res_line_4[2] + A + B + C
    
    res_line_5 = (A, B, C)
    print(res_line_5)
    
    res_line_6 = (Tot_el_cond, Tot_Seeb, Tot_ke)
    print(res_line_6)
    
    return res_line_0 + res_line_1 + res_line_2 + res_line_3 + res_line_4 + res_line_5 + res_line_6


#Test lines--------------
print(Point_n_type(800, 0.0, -1.0E18))
#------------------------

#Temperature
#x = np.linspace(600, 800, num = 1)
#print(x)
#Sn_content
#y = np.linspace(0.0, 0.0, num = 1)
#print(y)
#N_effective
#z = np.logspace(18,20, num = 1)
#z = np.array([0.52, 0.96, 1.68]#, 2.79, 4.04, 5.74, 7.91, 14.4])
#z = z*1.0E19
#print(z)


calculate = False
             
if calculate:
    headers = ['Temp', 'Sn_content', 'N_effective', 'chem_pot', 'n', 'lh', 'hh', 'n-sigma', 'n-alfa', 'n-ke', 'lh-sigma', 'lh-alfa', 'lh-ke', 'hh-sigma', 'hh-alfa', 'hh-ke', 'A', 'B', 'C', 'tot-sigma', 'tot-alfa', 'tot-ke']
    with open("out.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(headers)

    result = []
    
    for item_x in x:
        for item_y in y:
            for item_z in z:
                temp = Point_n_type(item_x, item_y, item_z)
                print(temp)
                with open("out.csv", "a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow(temp)
                result.append(temp)


(800, 0.0, -1e+17)
(-0.15751840608927395, 3.413479768893284e+18, 8.783699422233208e+17, 2.6351098266699597e+18)
4.0
(70.97982751030251, 390.08782789645306, 0.07862429346722177)
4.0
(12.542160124230325, 468.39426909202706, 0.008659738588154275)
12.0
(38.1327908820955, 467.44365365817293, 0.026298446104617734)
(0.005228672260840408, 0.011875026671127447, 6.823030372114705e-07)
(121.65477851662834, -32.78753015920206, 0.13068685939499886)
(800, 0.0, -1e+17, -0.15751840608927395, 3.413479768893284e+18, 8.783699422233208e+17, 2.6351098266699597e+18, 70.97982751030251, 390.08782789645306, 0.07862429346722177, 12.542160124230325, 468.39426909202706, 0.008659738588154275, 38.1327908820955, 467.44365365817293, 0.026298446104617734, 0.005228672260840408, 0.011875026671127447, 6.823030372114705e-07, 121.65477851662834, -32.78753015920206, 0.13068685939499886)


In [1]:
sample_name = ['p_00_1E18', 'p_10_2E18', 'p_25_13e18', 'n_00_5E18', 'n_00_10E18']

def generate_condition_sets(name_list):
    for name in name_list:
        name = name.split('_')
        print(name)